# Nbody_SH1

`nbody_SH1` is a stand-alone version of an Nbody system with shared time steps, based on the fourth order Hermite integration scheme.

The file `nbody_SH1.C` contains a C++ program, and can be compiled
directly as is, without any need to link to anything else.

### General scheme
The code is a **Direct Nbody code** which envolves solving sets of coupled differential equations, with some initial conditions set by the user.

Starting from the initial conditions, the code evolves the system as follows:
- Initialization
- Gravitational force computation
- Integration/Advance to the next timestep
- Set advance parameters as initial condition and repeat

**Timestepping is important**, it's the $\Delta t$ in which the simulation will be subdivided. You don't want it too small nor too large. 

In general, the code computes the total behaviour of a collection of particles by copmuting the *net force* acting on each of those particles, that is the sum for all the $j$ particles $\neq$ from the $i$ particle.

$$
    \vec f_{ij} = - \dfrac{Gm_im_j}{|\vec r_{ij}|^3}(\vec r_i - \vec r_j)
$$

In this scheme, the force is measured directly, which ensures the **best precision**, since the error is only due to the discretization of the distribution in N points.  

### Units
This code uses **internal units** so $G = 1$. To move from internal units to the cgs system we use an adimensional quantity as a vehicle of conversion.

In our case we choose to use the centripetal force - gravitational force equality for circular orbits. The earth-sun system can be approximated as such, therefore we will use that.
From the definition of *centripetal force*<br><br>
$$ f_\text{cp} = \dfrac{v^2}{r} = \dfrac{GM}{r^2}$$
<br>
we define the following *adimensional quantity* for our conversion, called **virial parameter**

$$
\bigg(\dfrac{GM}{rv^2}\bigg)_\text{iu} = \bigg(\dfrac{GM}{rv^2}\bigg)_\text{cgs} 
$$

Since the Newtonian attraction is scale invariant, we can choose whichever scale we like and fix two of the three variables to any value we want. e.g. mass and radius of a cluster, or in general anything we want to simulate. 
</br>
In this case, I decide to move from internal units to cgs by choosing $m_\text{iu} = 1 M_\odot$ and $r = 1 \text{pc}$, which in cgs converts to 
</br>
</br>
$$
\dfrac{m_\text{cgs}}{m_\text{iu}} = 2 \times 10^{33} \qquad \dfrac{r_\text{cgs}}{r_\text{iu}} = 3 \times 10^{18}
$$
</br>
From which we find, using $G = 6.67 \times 10^{-8}$ in the cgs units 
$$
\dfrac{v_\text{cgs}}{v_\text{iu}} = 6 \times 10^3 \ \text{cm}/\text{s}
$$

If i want my choice to be physically consistent i still need to choose mass and radius as property of the same system. In this case, the choice of a binary of 1 solar mass distant 1 pc is non physical.

### Input
The code expects an input file as initial conditions with the following structure:

- `N_particles` an integer with the total particles to simulate
- `current_time` the time at the beginning of the simulation
- `N_lines` a series of lines each with an array
```C++
(m, x, y, z, v_x, v_y, v_z)
```
where you can set the initial mass, position and velocity of each particle.

### The Hermite integration scheme
Given the ICs, the code computes the integration for one timestep using Taylor series up to fourth order, this is why we call it 4th order Hermite
$\\[.5em]$
$$
\begin{cases}
    x_1 = x_0 + v_0\Delta t + \dfrac{1}{2}a_0{\Delta t}^2 + \dfrac{1}{3!}j_0{\Delta t}^3 + \dfrac{1}{4!}s_1(t_0){\Delta t}^4 + O({\Delta t}^5) \hspace{2em} (1)\\[1.5em]
    v_1 = v_0 + a_0{\Delta t} + \dfrac{1}{2}j_0{\Delta t}^2 + \dfrac{1}{3!}s_0{\Delta t}^3 + O({\Delta t}^4) \hspace{7.5em} (2)\\[1.5em]
    a_1 = a_0 + j_0{\Delta t} + \dfrac{1}{2}s_0{\Delta t}^2 + O({\Delta t}^3) \hspace{11.8em}(3)\\[1.5em]
    j_1 = j_0 + s_0\Delta t + O({\Delta t}^2) \hspace{15.8 em}(4)\\[1.5em]
\end{cases}
$$
$\\[.5em]$

The underscore $1$ denotes the next timestep, that is $(t_0 + \Delta t)$. We use equations (3) and (4) to eliminate the first and the second derivatives of the jerk (snap and crackle) in equations 1 and 2 respectively.

So we compute $j_1$ and $a_1$ first and substitute them into (1) and (2), we get
$\\[.5em]$
$$
    x_1 = x_0 + \dfrac{1}{2}(v_0 + v_1)\Delta t + \dfrac{1}{12}(a_0 - a_1)\Delta t^2 + O(\Delta t^5) \hspace{5em} (5) \\[1em]
    v_1 = v_0 + \dfrac{1}{2}(a_0 + a_1)\Delta t + \dfrac{1}{12}(j_0 - j_1)\Delta t^2 + O(\Delta t^5) \hspace{5em} (6)
$$

which are 4th order accurate.

The problem is that $v_1, a_1, j_1$ are not known at the time of integration so we need to *predict* them from Newton's gravitation formula.

### Predictor step
We use 3rd order Taylor expansion to predict $x_1$ and $v_1$

$$
    x_p,1 = x_0 + v_0\Delta t + \dfrac{1}{2}a_0 \Delta t^2 + \dfrac{1}{6}j_0\Delta t^3 \\[1em]
    v_p,1 = v_0 + a_0 \Delta t + \dfrac{1}{2}j_0\Delta t^2
$$

### Force evaluation step
We use the predicitons to evaluate predicted values of $a_1$ and $j_1$ using Newton's Formula

$$
    \vec{a}_i = -G \sum_{j\neq i} \dfrac{M_j}{|\vec r_{ij}|^3}\ (\vec r_i - \vec r_j) 
$$

And we often add a **softening parameter** $\epsilon$ which normally would be the physical radius of stars, to ensure attenuation of the force at small radii 

$$
    \vec{a}_i = - G \sum_{j\neq i} \dfrac{M_j}{\left(r_{ij}^2 + \epsilon^2\right)^{3/2}}\ \vec r_{ij}
$$

The jerk will be the time derivative of that, so

$$
        \vec{j}_i = - G \sum_{j\neq i} M_j\left[\dfrac{\vec v_{ij}}{\left(r^2_{ji} + \epsilon^2\right)^{3/2}} + \dfrac{3(\vec v_{ij} \cdot \vec r_{ij})\vec r_{ij}}{\left(r^2_{ji} + \epsilon^2 \right)^{5/2}}\right]
$$

where $\vec v_{ij} = (\vec v_i - \vec v_j)$ and $r_{ji} = |\vec{r_j} - \vec{r_i}|$

### Correction step
We finally subsitute the predicetd acceleration and jerk into the equations (5) and (6). In particular we start from (6) 

$$
    v_1 = v_0 + \dfrac{1}{2}(a_0 + a_{p,1})\Delta t + \dfrac{1}{12}(j_0 - j_{p,1})\Delta t^2
$$

and then substitute this into $x_1$. This way we ensure the 4th order accuracy to position as well

$$
    x_1 = x_0 + \dfrac{1}{2}(v_0 + v_{1})\Delta t + \dfrac{1}{12}(a_0 - a_{p,1})\Delta t^2
$$

These are the initial conditions for the next timestep.

### Choice of timestep
We could use a fixed $\Delta t$ for all particles, but some undergo close encounters so the force changes much more rapidly than it does for other particles. We want different timesteps:
- longer for 'unperturbed' particles
- shorter for close encounters

A frequently used choice is **Block time steps**, each particle has
$$
    \large \Delta t_i = \eta\ \dfrac{a_i}{j_i} 
$$

where $\eta$ is called precision parameter, normally $\eta \sim 0.01-0.02$ is a good choice.
The **system time step** is defined as $\min(\Delta t_i)$, the smallest value from all particles timestep. 

Those with $\Delta t_i = \min(\Delta t_i)$ are called **active particles**.

Position and velocities are predicted for all particles.

Jerk and acceleration predictions and position and velocity corrections are done only for active particles.

### Block timestep
Since evaluating $\Delta t_i$ for each particle is very expensive, we substitute the individual timesteps with a block timestep such that $t/\Delta t_b$ is an integer.

### Running the command
```
./nbody_SH1 < data.in > data.out -d (step size control parameter) -e (diagnostics interval) -o (output interval) -t (total duration) -i (start output at t=t0)
```